## Финальный Feature Engineering + выгрузка в SQL

В этом ноутбуке формируются итоговые признаки и создаётся финальный датасет.
После этого датасет выгружается в PostgreSQL для дальнейшего анализа и BI-дашборда

Задачи ноутбука:

- сформировать логичные, интерпретируемые признаки

- подготовить таблицу для хранения в базе данных

- проверить, что данные корректные

- экспортировать их в PostgreSQL

Используем уже **очищенный** датасет `clean_churn.csv`, который был сформирован во втором ноутбуке.
Теперь никаких пропусков и проблемных строк нет, формат данных корректен.

In [1]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import psycopg2


df = pd.read_csv("../data/processed/clean_churn.csv")

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


В этом блоке формируются показатели, которые важны для анализа и принятия бизнес-решений

**contract_length:**

Признак, который превращает строковое поле **Contract** в количественный показатель длительности контракта:

- Month-to-month &#8594; 1

- One year &#8594; 12

- Two year &#8594; 24

**has_security:**

Создаётся бинарный признак, отражающий наличие **OnlineSecurity**:

- Yes &#8594; 1

- No &#8594; 0

- No internet service &#8594; 0

**has_techsupport:**

Аналогичный бинарный признак для TechSupport - ещё один важный фактор удержания.

**services_count:**

Полезный композитный признак:

Количество сервисов, которыми пользуется клиент

Складывается:

- OnlineSecurity

- OnlineBackup

- DeviceProtection

- TechSupport

- StreamingTV

- StreamingMovies

**churn_flag:**

Перевод категории Churn &#8594; 0/1

После формирования признаков выводится `df.info()`, чтобы убедиться, что все типы корректны.

In [2]:
df['contract_lenght']=df['Contract'].map({'Month-to-month':1, 'One year':12, 'Two year':24})

df['has_security']=df['OnlineSecurity'].map({'Yes':1, 'No':0, 'No internet service':0})

df['has_techsupport']=df['TechSupport'].map({'Yes':1, 'No':0, 'No internet service':0})

df['services_count'] = df[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']].replace({'Yes':1, 'No':0, 'No internet service':0}).sum(axis=1)

df['churn_flag']=df['Churn'].map({'Yes':1, 'No':0})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


**Подключение к PostgreSQL и загрузка данных:**

- загружаем переменные из .env

- создаём строку подключения postgresql+psycopg2

- создаём engine через SQLAlchemy

- выгружаем весь DataFrame в таблицу `churn_data`

Используем: `to_sql(if_exists='replace')`
Это гарантирует, что при повторном запуске таблица будет обновлена.

Также делается пробный SQL-запрос: `SELECT * FROM churn_data LIMIT 2;`

Это проверяет:

- подключение работает

- данные загружены

- структура таблицы корректна


In [3]:
import os
from sqlalchemy import create_engine, text
import pandas as pd
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)

PGHOST = os.getenv("PGHOST", "localhost")
PGPORT = os.getenv("PGPORT", "5432")
PGDATABASE = os.getenv("PGDATABASE", "churn_database")
PGUSER = os.getenv("PGUSER", "churn_user")
PGPASSWORD = os.getenv("PGPASSWORD", "strong_password")

conn_str = f"postgresql+psycopg2://{PGUSER}:{PGPASSWORD}@{PGHOST}:{PGPORT}/{PGDATABASE}"

engine = create_engine(conn_str, pool_size=5, max_overflow=10)

df.to_sql('churn_data', con=engine, if_exists='replace', index=False, method='multi', chunksize=1000)

q = "SELECT * FROM churn_data LIMIT 2;"
sample = pd.read_sql_query(q, con=engine)
print(sample.head())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   

      MultipleLines InternetService OnlineSecurity  ... PaperlessBilling  \
0  No phone service             DSL             No  ...              Yes   
1                No             DSL            Yes  ...               No   

      PaymentMethod MonthlyCharges TotalCharges Churn contract_lenght  \
0  Electronic check          29.85        29.85    No               1   
1      Mailed check          56.95      1889.50    No              12   

  has_security has_techsupport  services_count  churn_flag  
0            0               0               1           0  
1            1               0               2           0  

[2 rows x 26 columns]


Финальная таблица сохраняется локально как: `churn_final.csv`

In [4]:
df.to_csv("../data/processed/churn_final.csv", index = False)

### Итоги работы ноутбука

В результате выполнения ноутбука получаем:

- Финальный датасет, дополненный важными признаками

- Готовую таблицу в PostgreSQL для BI-дашборда

- Локальный CSV для дальнейшей работы

- Чистый, удобный, структурированный набор данных